In [14]:
import os
import json
import torch
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.path import Path
from torchvision.transforms import v2 as T
from torch.utils.data import DataLoader

from transformers import DetrImageProcessor, DetrForObjectDetection
import numpy as np


In [15]:
import os
import json
from PIL import Image
import torch
from torchvision.transforms import functional as F
from tqdm import tqdm

class CustomDataset:
    def __init__(self, folder_path, transforms=None):
        self.folder_path = folder_path
        self.transforms = transforms
        self.data_pairs = self._load_data_pairs()
        self.image_processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")

    def _load_data_pairs(self):
        image_files = []
        json_files = {}
        data_pairs = []

        # 폴더를 재귀적으로 검색하여 이미지 파일과 JSON 파일 목록을 생성
        for root, _, files in os.walk(self.folder_path):
            for file in files:
                if file.endswith((".jpg", ".jpeg", ".png")):
                    image_files.append(os.path.join(root, file))
                elif file.endswith(".json"):
                    json_files[os.path.splitext(file)[0]] = os.path.join(root, file)

        # 진행률 표시줄 추가
        for image_file_path in tqdm(image_files, desc="Matching image and JSON files"):
            # 이미지 파일 이름에서 확장자를 제외한 부분 가져오기
            image_name = os.path.splitext(os.path.basename(image_file_path))[0]
        
            # 해당 이미지 파일과 매칭되는 JSON 파일 찾기
            if image_name in json_files:
                json_file_path = json_files[image_name]
                data_pairs.append((image_file_path, json_file_path))
            else:
                print(f"JSON file not found for image: {image_file_path}")
        
        return data_pairs

    def create_detr_target(self, json_data):
        annotations = json_data['annotations']
        categories = {cat['id']: cat['name'] for cat in json_data['categories']}
        
        # 한글에서 영어로 라벨을 매핑하는 딕셔너리
        label_mapping = {'화방': 0, '줄기': 1, '잎': 2, '열매': 3}  # 예시, 실제 매핑에 맞게 수정 필요

        height = 1960  # 로즈 테스트시 고정값 사용
        width = float(json_data['images'][0]['width'])

        target = {
            'boxes': [],
            'labels': [],
            'area': [],
            'iscrowd': [],
        }

        for ann in annotations:
            bbox = ann['bbox']
            obj_name = categories[ann['category_id']]
            
            # 한글 카테고리 이름을 영어로 변환하여 매핑된 정수 레이블 얻기
            label = label_mapping.get(obj_name, -1)  # 없는 경우에 대한 처리 필요

            if label == -1:
                continue  # 처리할 수 없는 경우 스킵하거나 예외 처리 필요
            
            # 바운딩 박스 좌표 변환 및 정규화
            x_min = bbox[1] / width
            y_min = (height-(bbox[0]+bbox[2])) / height
            box_width = bbox[3] / width
            box_height = bbox[2] / height
            
            target['boxes'].append([x_min, y_min, box_width, box_height])
            target['area'].append(ann['area'])
            target['iscrowd'].append(ann['isCrowd'])
            target['labels'].append(label)

        # 리스트를 텐서로 변환
        target['boxes'] = torch.tensor(target['boxes'], dtype=torch.float32)
        target['area'] = torch.tensor(target['area'], dtype=torch.float32)
        target['iscrowd'] = torch.tensor(target['iscrowd'], dtype=torch.int64)
        target['labels'] = torch.tensor(target['labels'], dtype=torch.int64)  # 클래스 레이블을 정수로 변환

        return target

    def preprocess_image(self, image):
        img = self.image_processor(image, return_tensors="pt")
        images = img['pixel_values'].clone().detach().to(torch.float32)
        images = images.permute(0, 3, 1, 2)  # (batch_size, channels, height, width) 순으로 변환
        return images
        
    def __getitem__(self, index):
        img_path, json_path = self.data_pairs[index]

        with open(json_path, 'r') as f:
            json_data = json.load(f)

        img = Image.open(img_path).convert("RGB")

        # Preprocess the image using AutoImageProcessor
        images = self.preprocess_image(img)

        target = self.create_detr_target(json_data)

        return images, target

    def __len__(self):
        return len(self.data_pairs)


In [16]:
# 데이터셋과 변환 정의
train_path = 'path_to_train_data_folder'
val_path = 'path_to_validation_data_folder'
test_path = 'path_to_test_data_folder'

In [17]:
transform_train = T.Compose([
    T.Resize((800, 1333)),  # 이미지 크기 조정
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 이미지 정규화
])

transform_val = T.Compose([
    T.Resize((800, 1333)),  # 이미지 크기 조정
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 이미지 정규화
])

c:\Users\User\anaconda3\envs\py3_11\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [18]:
# 데이터셋 객체 생성
train_dataset = CustomDataset(train_path, transforms=transform_train)
val_dataset = CustomDataset(val_path, transforms=transform_val)
test_dataset = CustomDataset(test_path, transforms=None)
# 데이터 로더 설정
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

Matching image and JSON files: 0it [00:00, ?it/s]
Matching image and JSON files: 0it [00:00, ?it/s]
Matching image and JSON files: 0it [00:00, ?it/s]


ValueError: num_samples should be a positive integer value, but got num_samples=0